### COSINE SIMILARITY IN BIGQUERY 

In [4]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [5]:
USER_ID = 3709618 #900
LAST_DATA_TIME = "DATETIME '2018-12-31'"
DAYS_RECOMMEND = 5

In [6]:
import google.datalab.bigquery as bq

Saque la query de aca ->
[Analyzing text semantic similarity ]('https://cloud.google.com/solutions/machine-learning/analyzing-text-semantic-similarity-using-tensorflow-and-cloud-dataflow#explore_similar_articles_in_bigquery')

In [14]:
def create_cosine_similarity_query(user_id, number_of_articles):
  embed_columns = ["v{}".format(i) for i in range(50)]
  embed_columns_str = ",".join(embed_columns)
  average_columns_str = ",".join(map(lambda x: 'AVG({})'.format(x),embed_columns))
  query = """
      SELECT
      c.articles_key as similar_article_id,
      SUM(vv1*vv2) / (SQRT(SUM(POW(vv1,2))) * SQRT(SUM(POW(vv2,2)))) AS similarity,
      d.title,
      d.body,
      d.section_1,
      d.tag_1
    FROM
      (SELECT
         input_embed.val input_embed, articles_embed.key articles_key, articles_embed.val articles_embeds

        FROM (
          SELECT
              [{}] val
          FROM
            `AGEA_ASL.Dataset_D`
          WHERE user_id = '{}'
        )  input_embed
        CROSS JOIN
        (
          SELECT
          content_id key,
          [{}] val
          FROM `AGEA_ASL.Dataset_C`
          WHERE DATETIME_DIFF({}, CAST(day_write AS DATETIME),DAY) < {}
          AND content_id not in (SELECT content_id FROM `AGEA_ASL.Dataset_B` WHERE user_id='{}')
        ) articles_embed
    ) c
    , UNNEST(c.input_embed) vv1 with offset ind1 JOIN UNNEST(c.articles_embeds) vv2 with offset ind2 ON (ind1=ind2)
    LEFT JOIN `AGEA_ASL.Dataset_A` AS d ON c.articles_key = d.content_id
    GROUP BY c.articles_key,d.title,d.body,d.section_1,d.tag_1
    ORDER BY similarity DESC
    LIMIT {}
  """.format(average_columns_str,user_id, embed_columns_str,LAST_DATA_TIME,DAYS_RECOMMEND,user_id,number_of_articles)
  return query
  

In [15]:
print(create_cosine_similarity_query(USER_ID,4))


      SELECT
      c.articles_key as similar_article_id,
      SUM(vv1*vv2) / (SQRT(SUM(POW(vv1,2))) * SQRT(SUM(POW(vv2,2)))) AS similarity,
      d.title,
      d.body,
      d.section_1,
      d.tag_1
    FROM
      (SELECT
         input_embed.val input_embed, articles_embed.key articles_key, articles_embed.val articles_embeds

        FROM (
          SELECT
              [AVG(v0),AVG(v1),AVG(v2),AVG(v3),AVG(v4),AVG(v5),AVG(v6),AVG(v7),AVG(v8),AVG(v9),AVG(v10),AVG(v11),AVG(v12),AVG(v13),AVG(v14),AVG(v15),AVG(v16),AVG(v17),AVG(v18),AVG(v19),AVG(v20),AVG(v21),AVG(v22),AVG(v23),AVG(v24),AVG(v25),AVG(v26),AVG(v27),AVG(v28),AVG(v29),AVG(v30),AVG(v31),AVG(v32),AVG(v33),AVG(v34),AVG(v35),AVG(v36),AVG(v37),AVG(v38),AVG(v39),AVG(v40),AVG(v41),AVG(v42),AVG(v43),AVG(v44),AVG(v45),AVG(v46),AVG(v47),AVG(v48),AVG(v49)] val
          FROM
            `AGEA_ASL.Dataset_D`
          WHERE user_id = '3709618'
        )  input_embed
        CROSS JOIN
        (
          SELECT
          content_id 

In [16]:
def recommend_articles(user_id, number_of_articles):
  query = create_cosine_similarity_query(user_id, number_of_articles)
  return bq.Query(query).execute().result().to_dataframe()
  

In [17]:
def get_articles_readed_by_user(user_id, number_of_articles):
  query = """ 
    SELECT  
      a.title,
      a.body, 
      a.section_1,
      a.tag_1
    FROM `AGEA_ASL.Dataset_A` a 
    RIGHT JOIN `AGEA_ASL.Dataset_B` b ON a.content_id = b.content_id 
    WHERE b.user_id = '{}'
    LIMIT {}
  """.format(user_id, number_of_articles)
  return bq.Query(query).execute().result().to_dataframe()

### ARTICLES READED VS RECOMMENDED

In [18]:
articles_readed = get_articles_readed_by_user(USER_ID,100)
articles_readed.head(100)

,title,body,section_1,tag_1
0,El baile hipnótico de Mateo Messi que se viral...,desde hace un tiempo que mateo messi hijo del...,Deportes - Messi,Lionel Messi
1,Tragedia en el arranque de la temporada: cayer...,ocurrió en un segundo dicen quienes lo vieron...,Sociedad,Verano 2019
2,Tatiana creció en un hogar de menores y salió ...,abuso sexual abandono y maltrato son los ma...,Sociedad,Contámelo otra vez
3,Aníbal Pachano explotó contra Juan Darthés,la consigna era clara andy kusnetzoff le...,Espectáculos - Fama,Aníbal Pachano
4,El artista francés que pinta una medianera de ...,en el departamento número de alsina viv...,Sociedad,Arte
5,Vendía ropa en el living de su casa y ahora es...,hace diez años una vecina de san isidro comen...,Zonales,Diseño de moda
6,¡Inesperado llanto! Mirtha Legrand se quebró a...,un momento atípico se vivió en el programa de ...,Espectáculos - Fama,Mirtha Legrand
7,Cómo saber qué animal soy en el horóscopo chin...,rata búfalo tigre conejo dragón serpiente...,Entremujeres - Astrología,Horóscopo chino
8,Marixa Balli reveló terribles episodios de vio...,el estreno de el potro lo mejor del amor sacó...,Espectáculos - Fama,Rodrigo Bueno
9,Amorina Alday y Lucas Belbruno son los dos pri...,por decisión del público que los votó amorina...,Espectáculos - TV,La voz argentina


In [19]:
recommended_articles = recommend_articles(USER_ID, 20)
recommended_articles.head(20)

,similar_article_id,similarity,title,body,section_1,tag_1
0,5XJfNA3Cm,0.831555,"Dólar hoy: a cuánto cotiza, banco por banco",el dólar abrió este jueves con una nueva alza ...,Economía - Economía,Dólar hoy
1,SZS09KtDg,0.826800,Tragedia en un country: atropelló y mató a su ...,una mujer de años murió atropellada en un c...,Ciudades,Inseguridad vial
2,9Iia3ZDG0,0.804076,"Scaloni y la relación con Messi: ""Tenemos una ...",lionel scaloni el entrenador interino de la s...,Deportes - Selección Nacional,Lionel Scaloni
3,U0qSE9tjn,0.794347,El último novio de Juan Gabriel habló sobre el...,el último novio de juan gabriel habló y contó ...,Espectáculos - TV,Juan Gabriel
4,H-elKzT2H,0.771514,"Mike Amigorena: ""Un hombre me tocó en el subte...",el actor mike amigorena contó un episodio que ...,Espectáculos - TV,Mike Amigorena
5,LFlSFqkEY,0.764254,La pregunta sin filtro de Moria Casán a Sol Pé...,sol pérez dio una nota desde mar del plata do...,Espectáculos - TV,Sol Pérez
6,Bv4CciqDe,0.759313,"Video: como regalo de Navidad, le pidió a su p...",un joven que vive en houston estados unidos ...,Sociedad,Viste
7,01Q1POaVq,0.758964,"Carmen Barbieri y Santiago Bal, a los besos en...",a sala llena se estrenó nuevamente juntos la...,Espectáculos - Fama,Carmen Barbieri
8,DBgQw_zP_,0.756939,Barbie Vélez habló de la negativa de Calu Rive...,barbie vélez habló en intrusos américa sob...,Espectáculos - TV,Barbie Vélez
9,8HCJPe24n,0.756335,Escalofriante: Carolina Papaleo contó el maltr...,carolina papaleo habló a corazón abierto en i...,Espectáculos - TV,Carolina Papaleo
